# Evidently Metrics 

In [ ]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.options import DataDriftOptions
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DataDriftTable, DatasetDriftMetric, ColumnRegExpMetric

## Prepare a Dataset

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

## How to run Reports?

### Data Drift Metrics

In [ ]:
#dataset-level metrics
data_drift_dataset_report = Report(metrics=[
    DatasetDriftMetric(),
    DataDriftTable()    
])

data_drift_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_report

In [ ]:
#report in a JSON format
data_drift_dataset_report.json()

In [ ]:
#report as a python object
data_drift_dataset_report.as_dict()

In [ ]:
#column-level metrics
data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnDriftMetric('education'),
    ColumnDriftMetric('marital-status')
])

data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_report

### Data Integrity Metrics

In [ ]:
#column-level metrics
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*")
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

### How to set metric parameters?

In [ ]:
#simple metric parameters
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="education", reg_exp=r".*-.*", top=5),
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*")
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

In [ ]:
#options
stat_test_option = DataDriftOptions(all_features_stattest='psi')

data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnDriftMetric('age', options=stat_test_option),
])
data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)

data_drift_column_report